In [ ]:
from clique_benchmark import run_benchmark_sample, clique_oracle_compiler_classical_function, Trials, CompileType
from graph_database import Graphs, Graph

In [ ]:
graph_db = Graphs()
graph = graph_db.get(graph_id=51376)[0]
print(graph)
from clique_benchmark import _clique_size_to_search_for
print(_clique_size_to_search_for(graph, 1))


In [ ]:
run_benchmark_sample(
    graph=graph,
    clique_oracle_compiler=clique_oracle_compiler_classical_function,
    clique_size=_clique_size_to_search_for(graph, 1),
    compile_type=CompileType.CLASSICAL_FUNCTION,
    grover_iterations=1
)

In [ ]:
graph.graph_id

In [ ]:
target_grover_iterations = 2
for graph in graph_db.get(n=4, p=39):
    if _clique_size_to_search_for(graph, target_grover_iterations) > 1:
        print(graph.graph_id)
        print(graph)
        print(_clique_size_to_search_for(graph, target_grover_iterations))
        break